In [3]:
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification
import json


parameter= {
    "model": "microsoft/deberta-v3-large",
    "model_path": "model_e1_nb_12_unfreeze.pt",
    "max_length": 512,
    "batch_size": 5,
    "lr": 1e-3,
    "filter_no_pii_percent_allow": 0.1,
    "notebook": "11_weighted loss.ipynb"
}

def tokenize(example, tokenizer, max_length):
    text = []
    for t, ws in zip(example["tokens"], example["trailing_whitespace"]):
        text.append(t)
        if ws:
            text.append(" ")
    tokenized = tokenizer("".join(text), return_offsets_mapping=True, truncation=True, max_length=max_length, padding="max_length")
    text = "".join(text)
    length = len(tokenized.input_ids)
    return {
        **tokenized,
        "length": length,
    }

class PiiDatasetInference(torch.utils.data.Dataset):
    def __init__(self, dataset, tokenizer, max_length):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __getitem__(self, idx):
        vals=tokenize(self.dataset[idx], self.tokenizer, self.max_length)
        input_ids = torch.tensor(vals["input_ids"])
        attention_mask = torch.tensor(vals["attention_mask"])
        document_id = self.dataset[idx]["document_id"]
        return input_ids, attention_mask, document_id
    
    def __len__(self):
        return len(self.dataset)
    
class MyModel(torch.nn.Module):
    def __init__(self, model_name, num_labels, dropout_p=0.4):
        super().__init__()
        self.model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=num_labels)
        self.softmax=torch.nn.Softmax(dim=-1)
        
    def forward(self, input_ids, attention_mask):
        out=self.model(input_ids, attention_mask=attention_mask)['logits']
        out=self.softmax(out)
        return out

    
model=MyModel(parameter["model"], 13)
state_dict = torch.load(parameter['model_path'], map_location=torch.device('cpu'))
model.load_state_dict(state_dict, strict=False)

data = json.load(open("data/train.json"))
from itertools import chain
all_labels = sorted(list(set(chain(*[x["labels"] for x in data]))))
label2id = {l: i for i,l in enumerate(all_labels)}
id2label = {v:k for k,v in label2id.items()}

tokenizer = AutoTokenizer.from_pretrained(parameter["model"])
data = json.load(open("data/test.json"))
my_dataset=PiiDatasetInference(data, tokenizer, parameter["max_length"])
loader=torch.utils.data.DataLoader(my_dataset, batch_size=1, shuffle=True)
for id, attention_mask, document_id in loader:
    preds=model(id, attention_mask).argmax(dim=2)
    for pred, id in zip(preds.flatten(), id.flatten()):
        if pred != 12:
            print(f"Document: {document_id} TOKEN:{tokenizer.decode(id)}  --- pred:{id2label[pred.item()]}")
    
    
    

Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
c:\Users\Bernd\anaconda3\envs\mytorch\lib\site-packages\transformers\convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the 

KeyError: 'document_id'